# Proces ETL dla danych Airbnb  
1. Ekstrakcja danych, w tym określenie trybu dostępu do danych, identyfikacja anomalii, identyfikacja i obsługa zmian.  
2.	Transformacja danych, w tym, czyszczenie anomalii, uzgadnianie oraz standaryzacja danych,

In [116]:
import pandas as pd         

Wszystkie wykorzystane w ramach realizacji projektu dane zostały pobrane w postaci skompresowanych plików CSV ze strony interentowej http://insideairbnb.com/get-the-data 

### Wstępne wczytanie danych źródłowych w celu ich dalszego podziału na wymiary

Określenie trybu dostępu do danych:  

Pobrane ze strony dane mają postać plików CSV skompresowanych do pliku o rozszerzeniu .gz. Dla wymiaru listing, location, location_details oraz host jest to plik **listings.csv.gz.**   
Tryb dostępu: Otwieranie i odczytywanie pliku CSV przy użyciu biblioteki pandas metodą read_csv, z dodatkowym argumentem dekompresji jako gzip 

In [117]:
#wczytanie danych
df = pd.read_csv('listings.csv.gz', compression='gzip', dtype='object', usecols=['id','listing_url', 'name', 'description','host_id','host_url', 'host_name',
                                                                                  'host_since', 'host_location', 'host_response_time', 'host_is_superhost', 
                                                                                  'host_total_listings_count', 'host_identity_verified','room_type', 'accommodates',
                                                                                  'bedrooms','beds','amenities', 'price', 'minimum_nights','maximum_nights',
                                                                                  'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy',
                                                                                  'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication',
                                                                                  'review_scores_location', 'review_scores_value', 'neighbourhood_cleansed',
                                                                                  'neighbourhood_group_cleansed', 'longitude', 'latitude'])

df.head(3)

,id,listing_url,name,description,host_id,host_url,host_name,host_since,host_location,host_response_time,...,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,801749842377802394,https://www.airbnb.com/rooms/801749842377802394,A home away from home,The whole group will be comfortable in this sp...,495455523,https://www.airbnb.com/users/show/495455523,Michael,2023-01-10,NaN,NaN,...,2,30,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,765948794133787266,https://www.airbnb.com/rooms/765948794133787266,Brooklyn Refuge,Take a break and unwind at this peaceful oasis.,488760226,https://www.airbnb.com/users/show/488760226,Eric,2022-11-22,NaN,within an hour,...,2,1125,13,4.85,4.77,4.77,4.85,4.92,4.85,4.92
2,636274456676328779,https://www.airbnb.com/rooms/636274456676328779,Villa Masino.,Close to beach Peaceful walk to park & beach...,461263600,https://www.airbnb.com/users/show/461263600,Tommaso,2022-05-27,NaN,NaN,...,1,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Tworzymy dodatkowe kolumny umożliwiające łączenie między wymiarami - klucze obce

In [118]:
#tworzymy nowe kolumny - klucze obce do wymiarow location, neighbourhood
df['coordinates'] = df['latitude'].astype(str) + ',' + df['longitude'].astype(str)
df['location_id'] = df['coordinates'].astype('category').cat.codes
#usuwamy kolumny z wspolrzednymi
df = df.drop(['coordinates'], axis=1)

df['neighbourhood_id'] = df['neighbourhood_cleansed'].astype('category').cat.codes

df.head(3)

,id,listing_url,name,description,host_id,host_url,host_name,host_since,host_location,host_response_time,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,location_id,neighbourhood_id
0,801749842377802394,https://www.airbnb.com/rooms/801749842377802394,A home away from home,The whole group will be comfortable in this sp...,495455523,https://www.airbnb.com/users/show/495455523,Michael,2023-01-10,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2125,29
1,765948794133787266,https://www.airbnb.com/rooms/765948794133787266,Brooklyn Refuge,Take a break and unwind at this peaceful oasis.,488760226,https://www.airbnb.com/users/show/488760226,Eric,2022-11-22,NaN,within an hour,...,13,4.85,4.77,4.77,4.85,4.92,4.85,4.92,2859,29
2,636274456676328779,https://www.airbnb.com/rooms/636274456676328779,Villa Masino.,Close to beach Peaceful walk to park & beach...,461263600,https://www.airbnb.com/users/show/461263600,Tommaso,2022-05-27,NaN,NaN,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,197


### Wymiar listing

##### Ekstrakcja danych

In [119]:
#wczytanie danych
df_listings = df[['id','listing_url', 'name', 'description','room_type', 'accommodates','bedrooms','beds','amenities', 'price', 'minimum_nights','maximum_nights', 'number_of_reviews', 'review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication','review_scores_location', 'review_scores_value']]

df_listings.head(3)

,id,listing_url,name,description,room_type,accommodates,bedrooms,beds,amenities,price,minimum_nights,maximum_nights,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value
0,801749842377802394,https://www.airbnb.com/rooms/801749842377802394,A home away from home,The whole group will be comfortable in this sp...,Private room,2,1,1,"[""50\"" TV"", ""Bathtub"", ""Microwave"", ""Free driv...",$143.00,2,30,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,765948794133787266,https://www.airbnb.com/rooms/765948794133787266,Brooklyn Refuge,Take a break and unwind at this peaceful oasis.,Private room,1,1,1,"[""Free parking on premises"", ""Carbon monoxide ...",$30.00,2,1125,13,4.85,4.77,4.77,4.85,4.92,4.85,4.92
2,636274456676328779,https://www.airbnb.com/rooms/636274456676328779,Villa Masino.,Close to beach Peaceful walk to park & beach...,Entire home/apt,6,2,2,"[""BBQ grill"", ""Security cameras on property"", ...",$157.00,1,365,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [120]:
#wczytanie pliku sentymentow
df_sentiment = pd.read_csv('sentiment.csv', usecols=['listing', 'sentiment'])

df_sentiment.head()


,listing,sentiment
0,48295887,POSITIVE
1,23018280,POSITIVE
2,13397316,POSITIVE
3,2298373,POSITIVE
4,572446396370094272,POSITIVE


In [121]:
#grupujemy po id i agregujemy sentymenty jako count of positive i count of negative
df_sentiment = df_sentiment.groupby(['listing', 'sentiment']).size().reset_index(name='count')

df_sentiment.head(20)

,listing,sentiment,count
0,2595,NEGATIVE,1
1,2595,POSITIVE,3
2,5121,POSITIVE,2
3,5136,POSITIVE,1
4,5178,NEGATIVE,3
5,5178,POSITIVE,18
6,5203,POSITIVE,5
7,5586,POSITIVE,2
8,5803,POSITIVE,6
9,6848,POSITIVE,8


In [122]:
#tabela przestawna - pivot table
df_sentiment = df_sentiment.pivot(index='listing', columns='sentiment', values='count').reset_index()
#obsługa braków danych
df_sentiment = df_sentiment.fillna(0)

df_sentiment.head(10)

sentiment,listing,NEGATIVE,POSITIVE
0,2595,1.0,3.0
1,5121,0.0,2.0
2,5136,0.0,1.0
3,5178,3.0,18.0
4,5203,0.0,5.0
5,5586,0.0,2.0
6,5803,0.0,6.0
7,6848,0.0,8.0
8,6990,0.0,8.0
9,7097,1.0,9.0


In [123]:
#sprawdzamy duplikaty
df_sentiment.duplicated(subset=['listing']).sum()

0

In [124]:
#merge tabeli listings i sentiment
df_listings = df_listings.astype({'id': 'int64'})
df_listings = pd.merge(df_listings, df_sentiment, left_on='id', right_on='listing', how='left')
df_listings = df_listings.drop(['listing'], axis=1)
df_listings.head(10)

,id,listing_url,name,description,room_type,accommodates,bedrooms,beds,amenities,price,...,number_of_reviews,review_scores_rating,review_scores_accuracy,review_scores_cleanliness,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,NEGATIVE,POSITIVE
0,801749842377802394,https://www.airbnb.com/rooms/801749842377802394,A home away from home,The whole group will be comfortable in this sp...,Private room,2,1,1,"[""50\"" TV"", ""Bathtub"", ""Microwave"", ""Free driv...",$143.00,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,765948794133787266,https://www.airbnb.com/rooms/765948794133787266,Brooklyn Refuge,Take a break and unwind at this peaceful oasis.,Private room,1,1,1,"[""Free parking on premises"", ""Carbon monoxide ...",$30.00,...,13,4.85,4.77,4.77,4.85,4.92,4.85,4.92,0.0,1.0
2,636274456676328779,https://www.airbnb.com/rooms/636274456676328779,Villa Masino.,Close to beach Peaceful walk to park & beach...,Entire home/apt,6,2,2,"[""BBQ grill"", ""Security cameras on property"", ...",$157.00,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,768125251187660469,https://www.airbnb.com/rooms/768125251187660469,1-Bedroom Private Room with King Size Bed,Private room with king size bedroom near Sheep...,Private room,2,3,1,"[""Security cameras on property"", ""Keypad"", ""Ca...",$89.00,...,15,5.0,5.0,4.87,5.0,5.0,5.0,5.0,0.0,1.0
4,49248255,https://www.airbnb.com/rooms/49248255,Get the best of both worlds in Riverdale!,Welcome to the greatest location if you desire...,Entire home/apt,3,2,2,"[""Hangers"", ""Clothing storage: closet"", ""Secur...",$125.00,...,25,4.48,4.56,4.44,4.72,4.88,4.72,4.64,1.0,1.0
5,52059982,https://www.airbnb.com/rooms/52059982,Room 1,房间内带有两个标准的单人床，室内带有储藏室，宽敞舒适，氛围温馨。<br /><br /><b...,Private room,3,1,1,"[""Hangers"", ""Bathtub"", ""Microwave"", ""Hot water...",$63.00,...,40,4.68,4.88,4.55,4.78,4.78,4.93,4.68,0.0,1.0
6,783379064435522892,https://www.airbnb.com/rooms/783379064435522892,Welcome home,Disfruta de una experiencia con estilo en este...,Private room,2,1,1,"[""Air conditioning"", ""Free parking on premises...",$82.00,...,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,39125813,https://www.airbnb.com/rooms/39125813,Brand new construction nobody lived before 750sq,The apartment is brand new and nobody has live...,Entire home/apt,3,1,3,"[""Hangers"", ""Kitchen"", ""Shampoo"", ""Heating"", ""...",$118.00,...,3,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
8,654508528815045267,https://www.airbnb.com/rooms/654508528815045267,Room in Newly Renovated 4 Bedroom Townhome,"Private, air-conditioned room in a newly renov...",Private room,1,1,1,"[""Lock on bedroom door"", ""Air conditioning"", ""...",$49.00,...,1,5.0,5.0,5.0,5.0,5.0,5.0,5.0,NaN,NaN
9,6726337,https://www.airbnb.com/rooms/6726337,Beautiful 2 Bedroom Apartment!! NYC,Beautiful 2 bedroom Apartment!! it has everyth...,Entire home/apt,4,2,2,"[""Hangers"", ""Bathtub"", ""Microwave"", ""Dishwashe...",$115.00,...,16,5.0,4.93,4.93,4.93,4.93,4.8,4.73,NaN,NaN


Identyfikacja anomalii

In [125]:
#weryfikujemy czy dane posiadaja założoną wartość
for col in df_listings.columns:
    print(col, df_listings[col].unique())

id [801749842377802394 765948794133787266 636274456676328779 ...
           52491515           48158801           48316258]
listing_url ['https://www.airbnb.com/rooms/801749842377802394'
 'https://www.airbnb.com/rooms/765948794133787266'
 'https://www.airbnb.com/rooms/636274456676328779' ...
 'https://www.airbnb.com/rooms/52491515'
 'https://www.airbnb.com/rooms/48158801'
 'https://www.airbnb.com/rooms/48316258']
name ['A home away from home' 'Brooklyn Refuge' 'Villa Masino.' ...
 'Cozy 3 bedroom apt in the heart of Lower East Side'
 'Spacious Loft Space / Photo studio in Bushwick'
 'Sonder Flatiron | Spacious King Room+']
description ['The whole group will be comfortable in this spacious and unique space.<br /><br /><b>Guest access</b><br />you can enjoy the living room, dining room, kitchen and the backyard. The home has two entrances'
 'Take a break and unwind at this peaceful oasis.'
 'Close to beach   Peaceful walk to park & beach .close to shopping' ...
 "Location Location. Hell'

In [126]:
#sprawdzamy puste wartości
df_listings.isnull().sum()

id                                 0
listing_url                        0
name                              12
description                      753
room_type                          0
accommodates                       0
bedrooms                        3874
beds                             929
amenities                          0
price                              0
minimum_nights                     0
maximum_nights                     0
number_of_reviews                  0
review_scores_rating           10304
review_scores_accuracy         10739
review_scores_cleanliness      10729
review_scores_checkin          10743
review_scores_communication    10734
review_scores_location         10746
review_scores_value            10746
NEGATIVE                       28034
POSITIVE                       28034
dtype: int64

In [127]:
#sprawdzamy atrubuty
df_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42931 entries, 0 to 42930
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   id                           42931 non-null  int64  
 1   listing_url                  42931 non-null  object 
 2   name                         42919 non-null  object 
 3   description                  42178 non-null  object 
 4   room_type                    42931 non-null  object 
 5   accommodates                 42931 non-null  object 
 6   bedrooms                     39057 non-null  object 
 7   beds                         42002 non-null  object 
 8   amenities                    42931 non-null  object 
 9   price                        42931 non-null  object 
 10  minimum_nights               42931 non-null  object 
 11  maximum_nights               42931 non-null  object 
 12  number_of_reviews            42931 non-null  object 
 13  review_scores_ra

In [128]:
#sprawdzamy duplikaty
df_listings.duplicated().sum()

0

In [129]:
#check for outliers
df_listings.describe()


,id,NEGATIVE,POSITIVE
count,4.293100e+04,14897.000000,14897.000000
mean,2.222772e+17,0.204941,3.039740
std,3.344213e+17,0.529558,3.466663
min,2.595000e+03,0.000000,0.000000
25%,1.940474e+07,0.000000,1.000000
50%,4.337482e+07,0.000000,2.000000
75%,6.305016e+17,0.000000,4.000000
max,8.404660e+17,14.000000,66.000000


Transformacja danych

In [130]:
#zmiana nazw kolumny id na listing_id
df_listings = df_listings.rename(columns={'id': 'listing_id', 'NEGATIVE' : 'count_of_negative', 'POSITIVE' : 'count_of_positive'})

In [131]:
#obsluga brakujacych danych
df_listings['name'] = df_listings['name'].fillna('unknown')
df_listings['description'] = df_listings['description'].fillna('unknown')
df_listings['bedrooms'] = df_listings['bedrooms'].fillna(-1)
df_listings['beds'] = df_listings['beds'].fillna(-1)
df_listings['review_scores_rating'] = df_listings['review_scores_rating'].fillna(-1)
df_listings['review_scores_accuracy'] = df_listings['review_scores_accuracy'].fillna(-1)
df_listings['review_scores_cleanliness'] = df_listings['review_scores_cleanliness'].fillna(-1)
df_listings['review_scores_checkin'] = df_listings['review_scores_checkin'].fillna(-1)
df_listings['review_scores_communication'] = df_listings['review_scores_communication'].fillna(-1)
df_listings['review_scores_location'] = df_listings['review_scores_location'].fillna(-1)
df_listings['review_scores_value'] = df_listings['review_scores_value'].fillna(-1)
df_listings['count_of_negative'] = df_listings['count_of_negative'].fillna(-1)
df_listings['count_of_positive'] = df_listings['count_of_positive'].fillna(-1)

#sprawdzamy puste wartości
df_listings.isnull().sum()


listing_id                     0
listing_url                    0
name                           0
description                    0
room_type                      0
accommodates                   0
bedrooms                       0
beds                           0
amenities                      0
price                          0
minimum_nights                 0
maximum_nights                 0
number_of_reviews              0
review_scores_rating           0
review_scores_accuracy         0
review_scores_cleanliness      0
review_scores_checkin          0
review_scores_communication    0
review_scores_location         0
review_scores_value            0
count_of_negative              0
count_of_positive              0
dtype: int64

In [132]:
#usuwanie znaków specjalnych z kolumny price
df_listings['price'] = df_listings['price'].str.replace('$', '')
df_listings['price'] = df_listings['price'].str.replace(',', '')
#amenities
df_listings['amenities'] = df_listings['amenities'].str.replace('{', '')
df_listings['amenities'] = df_listings['amenities'].str.replace('}', '')
df_listings['amenities'] = df_listings['amenities'].str.replace('"', '')
df_listings['amenities'] = df_listings['amenities'].str.replace('/', '')
#description
df_listings['description'] = df_listings['description'].str.replace('<br />', '')
df_listings['description'] = df_listings['description'].str.replace('<b>', '')
df_listings['description'] = df_listings['description'].str.replace('</b>', '')
df_listings['description'] = df_listings['description'].str.replace('<p>', '')
df_listings['description'] = df_listings['description'].str.replace('</p>', '')
df_listings['description'] = df_listings['description'].str.replace('<strong>', '')
df_listings['description'] = df_listings['description'].str.replace('</strong>', '')
df_listings['description'] = df_listings['description'].str.replace('<br>', '')
df_listings['description'] = df_listings['description'].str.replace('<br/>', '')


C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\999651480.py:2: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_listings['price'] = df_listings['price'].str.replace('$', '')
C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\999651480.py:5: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_listings['amenities'] = df_listings['amenities'].str.replace('{', '')
C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\999651480.py:6: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_listings['amenities'] = df_listings['amenities'].str

In [133]:
#zmiana typu danych
numeric_columns = ['listing_id', 'accommodates', 'bedrooms', 'beds', 'minimum_nights', 'maximum_nights', 'number_of_reviews', 'count_of_negative', 'count_of_positive']
float_columns = ['price','review_scores_rating', 'review_scores_accuracy', 'review_scores_cleanliness', 'review_scores_checkin', 'review_scores_communication', 'review_scores_location', 'review_scores_value']
text_columns = ['listing_url', 'name', 'description', 'amenities', 'room_type']

def change_type_to_int(df, col):
    df[col] = df[col].astype('int64')
    return df

def change_type_to_float(df, col):
    df[col] = df[col].astype('float64')
    return df

def change_type_to_str(df, col):
    df[col] = df[col].astype(pd.StringDtype())
    return df

for col in numeric_columns:
    try:
        df_listings = change_type_to_int(df_listings, col)
    except:
        print(f"Nie udało się zmienić typu danych kolumny {col} na int64")

for col in text_columns:
    try: 
        df_listings = change_type_to_str(df_listings, col)
    except:
        print(f"Nie udało się zmienić typu danych kolumny {col} na String")
   
for col in float_columns:
    try: 
        df_listings = change_type_to_float(df_listings, col)
    except:
        print(f"Nie udało się zmienić typu danych kolumny {col} na float64")

df_listings.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 42931 entries, 0 to 42930
Data columns (total 22 columns):
 #   Column                       Non-Null Count  Dtype  
---  ------                       --------------  -----  
 0   listing_id                   42931 non-null  int64  
 1   listing_url                  42931 non-null  string 
 2   name                         42931 non-null  string 
 3   description                  42931 non-null  string 
 4   room_type                    42931 non-null  string 
 5   accommodates                 42931 non-null  int64  
 6   bedrooms                     42931 non-null  int64  
 7   beds                         42931 non-null  int64  
 8   amenities                    42931 non-null  string 
 9   price                        42931 non-null  float64
 10  minimum_nights               42931 non-null  int64  
 11  maximum_nights               42931 non-null  int64  
 12  number_of_reviews            42931 non-null  int64  
 13  review_scores_ra

In [134]:
#sprawdzamy czy nie ma danych numerycznych poza założonym przedziałem
def print_stats(df, column):
    stats = column.describe()
    return stats

for col in float_columns:
    print(col,"\n", print_stats(df_listings, df_listings[col]), "\n")

price 
 count    42931.000000
mean       200.307167
std        895.082911
min          0.000000
25%         75.000000
50%        125.000000
75%        200.000000
max      99000.000000
Name: price, dtype: float64 

review_scores_rating 
 count    42931.000000
mean         3.283620
std          2.487474
min         -1.000000
25%          1.000000
50%          4.700000
75%          4.950000
max          5.000000
Name: review_scores_rating, dtype: float64 

review_scores_accuracy 
 count    42931.000000
mean         3.311382
std          2.523072
min         -1.000000
25%         -1.000000
50%          4.790000
75%          5.000000
max          5.000000
Name: review_scores_accuracy, dtype: float64 

review_scores_cleanliness 
 count    42931.000000
mean         3.227317
std          2.484380
min         -1.000000
25%          0.000000
50%          4.640000
75%          4.930000
max          5.000000
Name: review_scores_cleanliness, dtype: float64 

review_scores_checkin 
 count    42931.0

In [135]:

#zapisz do pliku
df_listings.to_excel('dim_listings.xlsx', index=False,  engine='xlsxwriter')

### Wymiar location

Ekstrakcja danych

In [136]:
df_locations = df[['neighbourhood_id','neighbourhood_cleansed', 'neighbourhood_group_cleansed']]
df_locations.head()

,neighbourhood_id,neighbourhood_cleansed,neighbourhood_group_cleansed
0,29,Canarsie,Brooklyn
1,29,Canarsie,Brooklyn
2,197,Tottenville,Staten Island
3,177,Sheepshead Bay,Brooklyn
4,148,North Riverdale,Bronx


Identyfikacja anomalii

In [137]:
#weryfikujemy czy dane posiadaja założoną wartość
for col in df_locations.columns:
    print(col, df_locations[col].unique())

neighbourhood_id [ 29 197 177 148  78 214 159 166 113  13 162  72  68  56 170  22  44 158
 107  11  10 167 150 118 122 149  60  91  49  39 186 151 194   3  31   7
 196  80 209 120 183 101  40 189 130  82 168  99 164  55  15 152 185 192
  79 188 210 198  77 131 207 208 219  69  38   2  64 138  67  19 215 142
   9 216   4 201 175 155 111 108 134 105 187 136 125 212 157 112 205  93
   0  21 156  16  27 110   5 109 106 218 133  61  75  50  48  59 135  53
 119 126 137 191  86 222  17  62  98 203  65  96 171  47  12 153 160  52
  30 204 132 161  25 129  24  74  43  58 206  54  81 140  41 139  18  87
 184 115  28   6 199  35 200 180  33 211 117  76  97 121  94  37  88 147
 127 114  84  23 173 193 202 104 174  42 124 145 176  14 165  73  89 154
 181 163 217  66 172 195 178 220 190  63  45 146 103   1  36  70 100  46
 123 169  95  92 213  32  71 144  26  34  51   8 116 182  20  90  57 179
  85 143 128  83 102 221 141]
neighbourhood_cleansed ['Canarsie' 'Tottenville' 'Sheepshead Bay' 'North Rive

In [138]:
#sprawdzamy puste wartości
df_locations.isnull().sum()

neighbourhood_id                0
neighbourhood_cleansed          0
neighbourhood_group_cleansed    0
dtype: int64

In [139]:
#sprawdzamy duplikaty
df_locations.duplicated().sum()

42708

In [140]:
#data types
df_locations.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42931 entries, 0 to 42930
Data columns (total 3 columns):
 #   Column                        Non-Null Count  Dtype 
---  ------                        --------------  ----- 
 0   neighbourhood_id              42931 non-null  int16 
 1   neighbourhood_cleansed        42931 non-null  object
 2   neighbourhood_group_cleansed  42931 non-null  object
dtypes: int16(1), object(2)
memory usage: 754.8+ KB


Transformacja danych

In [141]:
df_locations = df_locations.drop_duplicates()
df_locations.duplicated().sum()

0

In [142]:
#reneame the columns
df_locations = df_locations.rename(columns={'neighbourhood_cleansed': 'neighbourhood', 'neighbourhood_group_cleansed': 'neighbourhood_group'})
#sort by neighbourhood_id
df_locations = df_locations.sort_values(by=['neighbourhood_id'])

df_locations.head()

,neighbourhood_id,neighbourhood,neighbourhood_group
266,0,Allerton,Bronx
5345,1,Arden Heights,Staten Island
168,2,Arrochar,Staten Island
73,3,Arverne,Queens
187,4,Astoria,Queens


In [143]:
#zmiana typu danych
text_columns = ['neighbourhood', 'neighbourhood_group']

for col in text_columns:
    df_locations = change_type_to_str(df_locations, col)

df_locations.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 223 entries, 266 to 577
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype 
---  ------               --------------  ----- 
 0   neighbourhood_id     223 non-null    int16 
 1   neighbourhood        223 non-null    string
 2   neighbourhood_group  223 non-null    string
dtypes: int16(1), string(2)
memory usage: 5.7 KB


In [144]:
#save the cleaned data
df_locations.to_excel('dim_locations.xlsx', index=False)


### Wymiar location details

Ekstrakcja danych

In [145]:
df_location_details = df[['location_id','latitude','longitude']]
df_location_details.head()

,location_id,latitude,longitude
0,2125,40.640402656712844,-73.88853475272201
1,2859,40.64851,-73.89433
2,2,40.507114,-74.251907
3,227,40.58349,-73.95988
4,41779,40.90326,-73.90709


Identyfikacja anomalii

In [146]:
for col in df_location_details.columns:
    print(col, df_location_details[col].unique())

location_id [ 2125  2859     2 ... 18949 10540 25349]
latitude ['40.640402656712844' '40.64851' '40.507114' ... '40.76318689388868'
 '40.76215' '40.68735']
longitude ['-73.88853475272201' '-73.89433' '-74.251907' ... '-73.98979'
 '-73.98517865966154' '-73.99247229211693']


In [147]:
#sprawdzamy duplikaty
df_location_details.duplicated().sum()

1138

In [148]:
#sprawdzamy puste wartości
df_location_details.isnull().sum()

location_id    0
latitude       0
longitude      0
dtype: int64

In [149]:
#sprawdzamy statystyki
df_location_details.describe()

,location_id
count,42931.000000
mean,21006.291701
std,12023.475623
min,0.000000
25%,10607.500000
50%,21136.000000
75%,31336.500000
max,41792.000000


In [150]:
#sprawdzamy typy danych
df_location_details.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42931 entries, 0 to 42930
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   location_id  42931 non-null  int32 
 1   latitude     42931 non-null  object
 2   longitude    42931 non-null  object
dtypes: int32(1), object(2)
memory usage: 838.6+ KB


Transformacja danych

In [151]:
#usuwamy duplikaty
df_location_details = df_location_details.drop_duplicates()
df_location_details.duplicated().sum()

0

In [152]:
#zmieniamy typ danych
df_location_details['location_id'] = df_location_details['location_id'].astype('int64')
df_location_details['latitude'] = df_location_details['latitude'].astype('float64')
df_location_details['longitude'] = df_location_details['longitude'].astype('float64')
df_location_details.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 41793 entries, 0 to 42930
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   location_id  41793 non-null  int64  
 1   latitude     41793 non-null  float64
 2   longitude    41793 non-null  float64
dtypes: float64(2), int64(1)
memory usage: 1.3 MB


In [153]:
float_columns = ['latitude', 'longitude']
for col in float_columns:
    print(col,"\n", print_stats(df_location_details, df_location_details[col]), "\n")

latitude 
 count    41793.000000
mean        40.727910
std          0.057938
min         40.500314
25%         40.687130
50%         40.723020
75%         40.762300
max         40.911380
Name: latitude, dtype: float64 

longitude 
 count    41793.000000
mean       -73.943276
std          0.056585
min        -74.251907
25%        -73.981320
50%        -73.952110
75%        -73.923670
max        -73.710870
Name: longitude, dtype: float64 



In [154]:
#zapisujemy dane do pliku
df_location_details.to_excel('dim_location_details.xlsx', index=False)


### Wymiar Host

Ekstrakcja danych

In [155]:
df_host = df[['host_id','host_url', 'host_name', 'host_since', 'host_location', 'host_response_time', 'host_is_superhost', 'host_total_listings_count', 'host_identity_verified']]

df_host.head()

,host_id,host_url,host_name,host_since,host_location,host_response_time,host_is_superhost,host_total_listings_count,host_identity_verified
0,495455523,https://www.airbnb.com/users/show/495455523,Michael,2023-01-10,NaN,NaN,f,1,f
1,488760226,https://www.airbnb.com/users/show/488760226,Eric,2022-11-22,NaN,within an hour,f,5,t
2,461263600,https://www.airbnb.com/users/show/461263600,Tommaso,2022-05-27,NaN,NaN,f,1,f
3,475699129,https://www.airbnb.com/users/show/475699129,Suliman,2022-08-18,NaN,within an hour,t,7,t
4,397288055,https://www.airbnb.com/users/show/397288055,Katherine,2021-04-16,NaN,within an hour,f,2,t


In [156]:
for col in df_host.columns:
    print(col, df_host[col].unique())


host_id ['495455523' '488760226' '461263600' ... '726333' '264962468' '6600525']
host_url ['https://www.airbnb.com/users/show/495455523'
 'https://www.airbnb.com/users/show/488760226'
 'https://www.airbnb.com/users/show/461263600' ...
 'https://www.airbnb.com/users/show/726333'
 'https://www.airbnb.com/users/show/264962468'
 'https://www.airbnb.com/users/show/6600525']
host_name ['Michael' 'Eric' 'Tommaso' ... 'Davina' 'Etiam' 'Scotty']
host_since ['2023-01-10' '2022-11-22' '2022-05-27' ... '2010-07-21' '2011-02-02'
 '2010-06-24']
host_location [nan 'New York, NY' 'United States' ... 'East Patchogue, NY' 'Bailey, CO'
 'Heredia, Costa Rica']
host_response_time [nan 'within an hour' 'a few days or more' 'within a few hours'
 'within a day']
host_is_superhost ['f' 't' nan]
host_total_listings_count ['1' '5' '7' '2' '20' '3' '21' '6' '9' '13' '10' '4' '8' '15' '562' '14'
 '16' '17' '28' '12' '283' '19' '34' '52' '246' '231' '11' '327' '2018'
 '58' '68' '23' '18' '94' '43' '342' '40' '127' 

In [157]:
#check duplicates of host_id
df_host.duplicated(subset=['host_id']).sum()

15476

In [158]:
#check for missing values
df_host.isnull().sum()

host_id                          0
host_url                         0
host_name                        5
host_since                       5
host_location                 9086
host_response_time           14532
host_is_superhost               25
host_total_listings_count        5
host_identity_verified           5
dtype: int64

In [159]:
#typy danych
df_host.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42931 entries, 0 to 42930
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype 
---  ------                     --------------  ----- 
 0   host_id                    42931 non-null  object
 1   host_url                   42931 non-null  object
 2   host_name                  42926 non-null  object
 3   host_since                 42926 non-null  object
 4   host_location              33845 non-null  object
 5   host_response_time         28399 non-null  object
 6   host_is_superhost          42906 non-null  object
 7   host_total_listings_count  42926 non-null  object
 8   host_identity_verified     42926 non-null  object
dtypes: object(9)
memory usage: 2.9+ MB


Transformacja danych

In [160]:
#delete duplicates
df_host.drop_duplicates(subset=['host_id'], inplace=True)
df_host.duplicated(subset=['host_id']).sum()

C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\2329217901.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_host.drop_duplicates(subset=['host_id'], inplace=True)


0

In [161]:
#handling missing values
df_host['host_name'] = df_host['host_name'].fillna('unknown')
df_host['host_location'] = df_host['host_location'].fillna('unknown')
df_host['host_response_time'] = df_host['host_response_time'].fillna('unknown')
df_host['host_total_listings_count'] = df_host['host_total_listings_count'].fillna(-1)

df_host.isnull().sum()

C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\3368295849.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_host['host_name'] = df_host['host_name'].fillna('unknown')
C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\3368295849.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_host['host_location'] = df_host['host_location'].fillna('unknown')
C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\3368295849.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

host_id                       0
host_url                      0
host_name                     0
host_since                    5
host_location                 0
host_response_time            0
host_is_superhost            25
host_total_listings_count     0
host_identity_verified        5
dtype: int64

In [162]:
#zmiana typu danych
text_columns = ['host_url', 'host_name', 'host_location', 'host_response_time']
int_columns = ['host_id', 'host_total_listings_count']

for col in int_columns:
    df_host = change_type_to_int(df_host, col)
    
for col in text_columns:
    df_host = change_type_to_str(df_host, col)

#zmiana typu danych na bool
df_host['host_is_superhost'] = df_host['host_is_superhost'].replace({'t': True, 'f': False})
df_host['host_identity_verified'] = df_host['host_identity_verified'].replace({'t': True, 'f': False})
df_host['host_is_superhost'] = df_host['host_is_superhost'].astype('bool')
df_host['host_identity_verified'] = df_host['host_identity_verified'].astype('bool')

#zmiana typu danych host_since
df_host['host_since'] = pd.to_datetime(df_host['host_since'])

df_host.info()

C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\4152442605.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype('int64')


<class 'pandas.core.frame.DataFrame'>
Int64Index: 27455 entries, 0 to 42929
Data columns (total 9 columns):
 #   Column                     Non-Null Count  Dtype         
---  ------                     --------------  -----         
 0   host_id                    27455 non-null  int64         
 1   host_url                   27455 non-null  string        
 2   host_name                  27455 non-null  string        
 3   host_since                 27450 non-null  datetime64[ns]
 4   host_location              27455 non-null  string        
 5   host_response_time         27455 non-null  string        
 6   host_is_superhost          27455 non-null  bool          
 7   host_total_listings_count  27455 non-null  int64         
 8   host_identity_verified     27455 non-null  bool          
dtypes: bool(2), datetime64[ns](1), int64(2), string(4)
memory usage: 1.7 MB


C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\4152442605.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[col] = df[col].astype(pd.StringDtype())
C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\705256887.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_host['host_is_superhost'] = df_host['host_is_superhost'].replace({'t': True, 'f': False})
C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\705256887.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a Data

In [163]:
float_columns = ['host_total_listings_count']
for col in float_columns:
    print(col,"\n", print_stats(df_host, df_host[col]), "\n")

host_total_listings_count 
 count    27455.000000
mean         4.972282
std         89.898471
min         -1.000000
25%          1.000000
50%          2.000000
75%          3.000000
max       8298.000000
Name: host_total_listings_count, dtype: float64 



In [164]:
#zapisanie danych do pliku
df_host.to_excel('dim_host.xlsx', index=False)


###  Tabela faktów- wynajem

Określenie trybu dostępu do danych:  

Pobrane ze strony dane mają postać plików CSV skompresowanych do pliku o rozszerzeniu .gz. Dla tabeli faktów jest to plik **calendar.csv.gz.**   
Tryb dostępu: Otwieranie i odczytywanie pliku CSV przy użyciu biblioteki pandas metodą read_csv, z dodatkowym argumentem dekompresji jako gzip 

Ekstrakcja danych

In [165]:
#wczytujemy dane
df_rents = pd.read_csv('calendar.csv.gz', compression='gzip', usecols=['listing_id', 'date', 'available', 'price'])

df_rents.head()

,listing_id,date,available,price
0,2595,2023-03-07,t,$150.00
1,2595,2023-03-08,t,$150.00
2,2595,2023-03-09,t,$150.00
3,2595,2023-03-10,f,$150.00
4,2595,2023-03-11,f,$150.00


In [166]:
for col in df_rents.columns:
    print(col, df_rents[col].unique())

listing_id [              2595               5121               5136 ...
 840150322371056021 840228878772481467 840466047136140141]
date ['2023-03-07' '2023-03-08' '2023-03-09' '2023-03-10' '2023-03-11'
 '2023-03-12' '2023-03-13' '2023-03-14' '2023-03-15' '2023-03-16'
 '2023-03-17' '2023-03-18' '2023-03-19' '2023-03-20' '2023-03-21'
 '2023-03-22' '2023-03-23' '2023-03-24' '2023-03-25' '2023-03-26'
 '2023-03-27' '2023-03-28' '2023-03-29' '2023-03-30' '2023-03-31'
 '2023-04-01' '2023-04-02' '2023-04-03' '2023-04-04' '2023-04-05'
 '2023-04-06' '2023-04-07' '2023-04-08' '2023-04-09' '2023-04-10'
 '2023-04-11' '2023-04-12' '2023-04-13' '2023-04-14' '2023-04-15'
 '2023-04-16' '2023-04-17' '2023-04-18' '2023-04-19' '2023-04-20'
 '2023-04-21' '2023-04-22' '2023-04-23' '2023-04-24' '2023-04-25'
 '2023-04-26' '2023-04-27' '2023-04-28' '2023-04-29' '2023-04-30'
 '2023-05-01' '2023-05-02' '2023-05-03' '2023-05-04' '2023-05-05'
 '2023-05-06' '2023-05-07' '2023-05-08' '2023-05-09' '2023-05-10'
 '202

In [167]:
df_rents.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15664741 entries, 0 to 15664740
Data columns (total 4 columns):
 #   Column      Dtype 
---  ------      ----- 
 0   listing_id  int64 
 1   date        object
 2   available   object
 3   price       object
dtypes: int64(1), object(3)
memory usage: 478.1+ MB


In [168]:
df_rents.isnull().sum()

listing_id       0
date             0
available        0
price         1112
dtype: int64

In [169]:
df_rents.duplicated().sum()

0

Transformacja danych

In [170]:
#obsluga brakujacych danych
df_rents['price'] = df_rents['price'].fillna(-1)
df_rents.isnull().sum()

listing_id    0
date          0
available     0
price         0
dtype: int64

In [171]:
#zmiana typu danych
df_rents['date'] = pd.to_datetime(df_rents['date'])
df_rents['price'] = df_rents['price'].str.replace('$', '').str.replace(',', '').astype(float)
df_rents['available'] = df_rents['available'].map({'t': True, 'f': False})
df_rents['available'] = df_rents['available'].astype('bool')
df_rents.info()

C:\Users\dabro\AppData\Local\Temp\ipykernel_4328\549253080.py:3: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_rents['price'] = df_rents['price'].str.replace('$', '').str.replace(',', '').astype(float)


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15664741 entries, 0 to 15664740
Data columns (total 4 columns):
 #   Column      Dtype         
---  ------      -----         
 0   listing_id  int64         
 1   date        datetime64[ns]
 2   available   bool          
 3   price       float64       
dtypes: bool(1), datetime64[ns](1), float64(1), int64(1)
memory usage: 373.5 MB


In [172]:
float_columns = ['price']
for col in float_columns:
    print(col,"\n", print_stats(df_rents, df_rents[col]), "\n")

price 
 count    1.566363e+07
mean     2.191347e+02
std      7.523764e+02
min      0.000000e+00
25%      8.000000e+01
50%      1.350000e+02
75%      2.250000e+02
max      1.000000e+05
Name: price, dtype: float64 



In [173]:
df_rents

,listing_id,date,available,price
0,2595,2023-03-07,True,150.0
1,2595,2023-03-08,True,150.0
2,2595,2023-03-09,True,150.0
3,2595,2023-03-10,False,150.0
4,2595,2023-03-11,False,150.0
...,...,...,...,...
15664736,840466047136140141,2024-03-01,False,100.0
15664737,840466047136140141,2024-03-02,False,100.0
15664738,840466047136140141,2024-03-03,False,100.0
15664739,840466047136140141,2024-03-04,False,100.0


In [174]:
#merge data from df_rents and df to get location_id, listing_id, host_id, neighbourhood_id where  listing_id = id
df['id'] = df['id'].astype('int64')
df_rents = pd.merge(df_rents, df[['id','location_id',  'host_id', 'neighbourhood_id']], left_on='listing_id', right_on='id', how='left')
df_rents = df_rents.drop(columns=['id'])
df_rents.head()


,listing_id,date,available,price,location_id,host_id,neighbourhood_id
0,2595,2023-03-07,True,150.0,28702,2845,129
1,2595,2023-03-08,True,150.0,28702,2845,129
2,2595,2023-03-09,True,150.0,28702,2845,129
3,2595,2023-03-10,False,150.0,28702,2845,129
4,2595,2023-03-11,False,150.0,28702,2845,129


In [175]:
df_rents['host_id'] = df_rents['host_id'].astype('int64')
df_rents.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 15664741 entries, 0 to 15664740
Data columns (total 7 columns):
 #   Column            Dtype         
---  ------            -----         
 0   listing_id        int64         
 1   date              datetime64[ns]
 2   available         bool          
 3   price             float64       
 4   location_id       int32         
 5   host_id           int64         
 6   neighbourhood_id  int16         
dtypes: bool(1), datetime64[ns](1), float64(1), int16(1), int32(1), int64(2)
memory usage: 702.1 MB


In [176]:
#save the cleaned data
df_rents.to_csv('fact_rents.csv', index=False)


### Wymiar Availability

Określenie trybu dostępu do danych:  

Manualnie stworzony wymiar na potrzeby modelu wymiarowego.

In [177]:
#stworzenie dwukolumnowej tabeli
availability = {'availability': ['available', 'not available'], 'available': [True, False]}
df_available = pd.DataFrame(availability)

#zmiana typu danych
df_available['availability'] = df_available['availability'].astype(pd.StringDtype())
df_available['available'] = df_available['available'].astype('bool')
df_available.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2 entries, 0 to 1
Data columns (total 2 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   availability  2 non-null      string
 1   available     2 non-null      bool  
dtypes: bool(1), string(1)
memory usage: 146.0 bytes


In [178]:
#save the cleaned data
df_available.to_excel('dim_availability.xlsx', index=False)
